In [24]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
import os
from google.cloud import pubsub_v1
from apache_beam import window
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [26]:
GCP_PROJECT_ID=os.getenv('GCP_PROJECT_ID')

# Replace 'my-service-account-path' with your service account path
SERVICE_ACCOUNT_KEY_PATH = "key.json"


print("Service account file : ", SERVICE_ACCOUNT_KEY_PATH)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SERVICE_ACCOUNT_KEY_PATH

PROJECT_ID = GCP_PROJECT_ID
TOPIC_ID = "poc-apache-beam"

# Initialize the Pub/Sub Publisher Client
publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(PROJECT_ID, TOPIC_ID)



Service account file :  key.json


In [37]:
# Function to read lines from the CSV file in streaming mode
def read_csv(_):
    with open("data/counts.csv", "r") as file:
        next(file)  # Skip the header
        for line in file:
            yield line.strip()  # Send one line at a time

# Define the pipeline
p = beam.Pipeline(options=options)


class BatchMessagePubSub(beam.DoFn):
    # Initialize the Pub/Sub Publisher Client
    publisher = pubsub_v1.PublisherClient()
    topic_path = publisher.topic_path(PROJECT_ID, TOPIC_ID)
    # Publish a message
    def publish_message(data):
        """
        Publish a message to the specified Pub/Sub topic.

        Args:
        - data (str): The message to be published.
        """
        # Convert message to bytes
        message_data = data.encode("utf-8")
        
        # Publish message
        future = publisher.publish(topic_path, message_data)
        message_id = future.result()  # Wait for the publish to complete
    
    def process(self, line):
        event_data = line.decode("utf-8").strip()
        self.publish_message(event_data)


In [ ]:
GCP_PROJECT_ID=os.getenv('GCP_PROJECT_ID')
# projects/<project>/topics/<topic>
TOPIC_ADDRESS = f"projects/{GCP_PROJECT_ID}/topics/poc-apache-beam"

options = PipelineOptions()
options.view_as(StandardOptions).streaming = True

p = beam.Pipeline(options=options)


pubsub_data = (
    p 
    | "Read from text file" >> beam.io.ReadFromText('data/counts.csv', skip_header_lines=1)
    | "Reshuffle to Prevent Batch Optimization" >> beam.Reshuffle()  # Creates an unbounded source effect
    | "Convert to Bytes" >> beam.Map(lambda row: row.encode("utf-8"))
    | 'Write to pus sub' >> beam.ParDo(BatchMessagePubSub())
                
)

result = p.run()
result.wait_until_finish()
